## Presentation Type Classifier and Networks


### What can You do with this Notebook

- Find and classify imitative points (fugas, imitative duos, and periodic entries)
- Build a network of patterns found in several pieces, and reveal 'communities' of compositions


#### Network Visualization with Louvain Communities

* Read more about **Louvain Community Detection Algorithms**:  https://towardsdatascience.com/community-detection-algorithms-9bd8951e7dae

* **Updated Summer 2022**
* **Key Features**:

    * Uses `getDistance` to identify `close matches` with side-by-side comparison of soggetti.  
    * For example: With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same. These are reported as "flexed entries" in a separate column.
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Labels Fuga, PEn, and ID according to time intervals between entries. NIm not yet supported!
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)
    
* Read the documentation:  `print(piece.presentationTypes.__doc__)`

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


## A.  Start by Finding Presentation Types in a Corpus of Pieces 

* The **CorpusBase** class is a convenient way to find patterns in any given list of pieces.
* The pieces are provided as a **list**, within square brackets and separated by commas.  
* The bracketed list is then contained within the parentheses of `CorpusBase()`
* For example: 

>`prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'
piece_list = ['CRIM_Model_0019',
             'CRIM_Mass_0019_1',
             'CRIM_Mass_0019_2',
             'CRIM_Mass_0019_3',
             'CRIM_Mass_0019_4',
             'CRIM_Mass_0019_5']
suffix = '.mei'
corpus_list  = []
for piece in piece_list:
url = prefix + piece + suffix
    corpus_list.append(url)
corpus = CorpusBase(corpus_list)` 
       
* Read the documentation:  `print(CorpusBase.batch.__doc__)`



In [2]:
prefix = 'https://crimproject.org/mei/'

piece_list = ['CRIM_Model_0019', 'CRIM_Mass_0019_1']
suffix = '.mei'



In [3]:
import glob
corpus_list = []
for name in glob.glob('Music_Files/*'):
    corpus_list.append(name)
corpus = CorpusBase(corpus_list)

Successfully imported Music_Files/Gascoigne_Deus.musicxml
Successfully imported Music_Files/Sermisy_Missa_Plumi_3.musicxml
Successfully imported Music_Files/Sermisy_Missa_Plumi_2.musicxml
Successfully imported Music_Files/Sermisy_Missa_Plumi_4.musicxml
Successfully imported Music_Files/Sermisy_Missa_Plumi_5.musicxml
Successfully imported Music_Files/Sermisy_Missa_Plumi_1.musicxml
Successfully imported Music_Files/Fevin_Benedictus.musicxml
Successfully imported Music_Files/Sermisy_Impetum.musicxml
Successfully imported Music_Files/Conseil_Adjuva.musicxml


### Now Run the PresentationType method on the Corpus

In [4]:
# This will need to be modified if using a corpus method different from the CRIM Project above.
final = pd.DataFrame()

for work in corpus_list:
#     file_url = prefix + piece 
#     url = prefix + work + suffix
    piece = importScore(work)   
    output = piece.presentationTypes(limit_to_entries = True,
                                     head_flex = 1,
                                      body_flex = 0,
                                    include_hidden_types = False,
                                    combine_unisons = True,
                                   melodic_ngram_length = 4)

    final['Validation'] = ""
    final['Comments'] = ""
    final = final.append(output, ignore_index=True)

# final.to_csv("corpus_classified.csv")
# final.to_csv("saved_csv/CRIM_Mass_0019_P_Types_hidden.csv")

Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.


In [5]:
final


,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,Parallel_Entries,Parallel_Voice,Count_Non_Overlaps
0,,,Mathieu Gascongne,Deus regnorum,140.0,"[18/1.0, 18/3.0, 23/3.0, 24/3.0]","[P5, P-12, P5]","[140.0, 144.0, 184.0, 192.0]","[(-2, -3, 2, 2)]","[4.0, 40.0, 8.0]","[[Contratenor], [Superius], [Bassus], [Tenor]]",FUGA,4.0,False,0.0,None,0.0
1,,,Mathieu Gascongne,Deus regnorum,256.0,"[32/3.0, 36/3.0]",[P1],"[256.0, 288.0]","[(2, 2, -3, 2)]",[32.0],"[[Contratenor], [Contratenor]]",FUGA,2.0,False,0.0,None,1.0
2,,,Mathieu Gascongne,Deus regnorum,322.0,"[40/4.0, 41/2.0]",[P-4],"[322.0, 326.0]","[(3, -2, -2, 2), (2, -2, -2, 2)]",[4.0],"[[Tenor], [Bassus]]",FUGA,2.0,True,0.0,None,0.0
3,,,Mathieu Gascongne,Deus regnorum,370.0,"[46/4.0, 48/2.0]",[P5],"[370.0, 382.0]","[(-2, -2, -2, -2)]",[12.0],"[[Bassus], [Tenor]]",FUGA,2.0,False,0.0,None,0.0
4,,,Mathieu Gascongne,Deus regnorum,414.0,"[52/2.0, 53/4.0]",[P-5],"[414.0, 426.0]","[(-3, 2, 2, -3)]",[12.0],"[[Superius], [Contratenor]]",FUGA,2.0,False,0.0,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,NaN,NaN,Jean Conseil\nCopied from Psalmorum Selectorum...,Adiuva me Domine,612.0,"[75/3.0, 77/3.0]",[M9],"[612.0, 628.0]","[(3, -2, 2, -2)]",[16.0],"[Tenor, Superius]",FUGA,2.0,False,0.0,None,0.0
95,NaN,NaN,Jean Conseil\nCopied from Psalmorum Selectorum...,Adiuva me Domine,682.0,"[84/2.0, 85/4.0]",[P-4],"[682.0, 694.0]","[(3, -2, -2, 2)]",[12.0],"[Superius, Tenor]",FUGA,2.0,False,0.0,None,0.0
96,NaN,NaN,Jean Conseil\nCopied from Psalmorum Selectorum...,Adiuva me Domine,714.0,"[88/2.0, 90/4.0, 96/2.0, 98/4.0]","[P-8, P8, P-8]","[714.0, 734.0, 778.0, 798.0]","[(2, -2, 4, -2)]","[20.0, 44.0, 20.0]","[Superius, Tenor, Superius, Tenor]",ID,4.0,False,0.0,None,0.0
97,NaN,NaN,Jean Conseil\nCopied from Psalmorum Selectorum...,Adiuva me Domine,732.0,"[90/3.0, 98/3.0]",[P1],"[732.0, 796.0]","[(2, 2, 2, -2)]",[64.0],"[Bassus, Bassus]",FUGA,2.0,False,0.0,None,1.0


#### Various Options to Filter

* by type
* by piece
* according to length between entries

In [6]:
# filter for only ID or PEN
pen_id_filter = final['Presentation_Type'].isin(["PEN"])
pen_id_only = final[pen_id_filter]
final = pen_id_only.copy()
len(final)



3

## B. Network Visualization with Louvain Communities

* This will only make sense for a corpus!

## Add Combined Data for Melodic, Time and Soggetti as "Words" for Network Edges

* We combine the lists of time and melodic intervals of entries as strings (thus a kind of word that uniquely identifies that pattern)
* We do the same for soggetti, combining the melodic intervals themselves as a string
* These are in turn assembled in various combinations:  TINT, MINT, and SOG, also as strings

In [7]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)

final['SOG'] = final['Soggetti'].apply(clean_melody_new)
final['ALL'] = final["MINT"] + '_' + final["TINT"] + '_' + final['SOG']

final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
final["MINT_SOG"] = final["MINT"] + '_' + final["SOG"]
final["TINT_SOG"] = final["TINT"] + '_' + final["SOG"]
final.head()

,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,...,Parallel_Entries,Parallel_Voice,Count_Non_Overlaps,MINT,TINT,SOG,ALL,ALL_INT,MINT_SOG,TINT_SOG
7,,,Mathieu Gascongne,Deus regnorum,656.0,"[81/3.0, 84/1.0, 86/3.0]","[P8, P-8]","[656.0, 676.0, 696.0]","[(-4, 2, 2, 2)]","[20.0, 20.0]",...,0.0,None,0.0,P8_P-8,20.0_20.0,-4_2_2_2,P8_P-8_20.0_20.0_-4_2_2_2,P8_P-8_20.0_20.0,P8_P-8_-4_2_2_2,20.0_20.0_-4_2_2_2
52,,,Claudin de Sermisy,Missa Plurium motettorum: Agnus Dei,4.0,"[1/1.0, 3/1.0, 5/1.0]","[P5, P8]","[4.0, 12.0, 20.0]","[(2, -2, -2, -2), (3, -2, -2, -2)]","[8.0, 8.0]",...,0.0,None,0.0,P5_P8,8.0_8.0,2_-2_-2_-2,P5_P8_8.0_8.0_2_-2_-2_-2,P5_P8_8.0_8.0,P5_P8_2_-2_-2_-2,8.0_8.0_2_-2_-2_-2
77,,,Claudin de Sermisy,Impetum inimicorum,24.0,"[3/1.0, 5/1.0, 7/1.0]","[P8, P-8]","[24.0, 40.0, 56.0]","[(2, -2, -3, 2), (3, -2, -3, 2)]","[16.0, 16.0]",...,0.0,None,0.0,P8_P-8,16.0_16.0,2_-2_-3_2,P8_P-8_16.0_16.0_2_-2_-3_2,P8_P-8_16.0_16.0,P8_P-8_2_-2_-3_2,16.0_16.0_2_-2_-3_2


### Make Groups, Based on Selected Criteria:

* MINT
* TINT
* SOG
* Or any of the combinations:
    * ALL (MINT + TINT + Soggetti)
    * ALL_INT (MINT + TINT)
    * MINT_SOG (MINT + Soggetti)
    * TINT_SOG (TINT + Soggetti)

In [8]:

output_grouped = final.groupby(['Title'])['MINT'].apply(list).reset_index()
output_grouped


,Title,MINT
0,Deus regnorum,[P8_P-8]
1,Impetum inimicorum,[P8_P-8]
2,Missa Plurium motettorum: Agnus Dei,[P5_P8]


### Group and Find Unique Pairs

- the pairing must be applied to columns that have more than one value, like Title, since these form the edges

In [9]:
pairs = output_grouped.MINT.apply(lambda x: list(combinations(x, 2)))
pairs2 = pairs.explode().dropna()
unique_pairs = pairs.explode().dropna().unique()

In [10]:
len(unique_pairs)

0

In [11]:
pd.Series(unique_pairs).isna().sum()

0

In [12]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

def create_node_html(node: str, source_df: pd.DataFrame, node_col: str):
    rows = source_df.loc[source_df[node_col] == node].itertuples()
    
    html_lis = []
    
    for r in rows:
        html_lis.append(f"""<li>Composer: {r.Composer}<br>
                                Title: {r.Title}<br>
                                Offset: {r.First_Offset}</li>"""
                       )
        
    html_ul = f"""<ul>{''.join(html_lis)}</ul>"""
        
    return html_ul


def add_nodes_from_edgelist(edge_list: list, 
                               source_df: pd.DataFrame, 
                               graph: nx.Graph,
                               node_col: str):
    
    graph = deepcopy(graph)
    
    node_list = pd.Series(edge_list).apply(pd.Series).stack().unique()
    
    for n in node_list:
        graph.add_node(n, title=create_node_html(n, source_df, node_col))
        
    return graph

In [13]:
def choose_network(df, chosen_word, file_name):
    
    output_grouped = df.groupby(['Title'])[chosen_word].apply(list).reset_index()
    pairs = output_grouped[chosen_word].apply(lambda x: list(combinations(x, 2)))
    pairs2 = pairs.explode().dropna()
    unique_pairs = pairs.explode().dropna().unique()


    pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="black", font_color="white")
    G = nx.Graph()
    
    try:
        G = add_nodes_from_edgelist(edge_list=unique_pairs, source_df=final, graph=G, node_col=chosen_word)
    except Exception as e:
        print(e)
    
    
    G.add_edges_from(unique_pairs)
    G = add_communities(G)
    pyvis_graph.from_nx(G)
    pyvis_graph.show(file_name)

In [14]:
# Find your rendered network in the Jupyter Hub file list!

choose_network(final, 'MINT', 'Mod_Mass_Full_Sog_19.html')

'Series' object has no attribute 'stack'
